In [199]:
%run data_plot.ipynb

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [200]:
'''
initialize the variables for storing possible routes and the 
best route for visualization and saving in a solutions file
'''
sim_anneal_data = {'best_path': {},
            'possible_paths': {}}
solutions = [{}]

In [201]:
def calculate_route_costs(coords_data):
    """
    calculate_route_costs(coords_data)
    calculates the bandwidth between nodes and each other. This is done using 
    the calculate_distance() and convert_distance_bandwidth() functions of the 
    data_plot module and added to a 2D array
    coords_data: the x, y coordinate data
    returns: the calculated bandwidth information for the various paths
    """
    bandwidth_data = []

    for data_row in coords_data.keys():
        # var for storing all path bandwidths & latencies
        path_costs = []
        for data_col in coords_data.keys():
            if data_row != data_col:
                # get path distance
                path_distance = calculate_distance(coords_data[data_row], coords_data[data_col])
                #print(f"Node {data_row} to Node {data_col}: {path_distance}")
                # get bandwidth between node and its next node and append it to path bandwidths var
                path_bandwidth = convert_distance_bandwidth(path_distance)
                #if path_bandwidth == 0:
                    #absolute_path_cost = 0
                #else:
                    #absolute_path_cost = round(absolute_cost_function(path_bandwidth, node_to_node_latency), 3)
                path_costs.append(path_bandwidth)
            else:
                path_costs.append(0)
        bandwidth_data.append(path_costs)

    return bandwidth_data

In [202]:
# Define the objective function to be optimized
def generate_route(origin, bandwidth_data):
    """
    generate_route(origin)
    generates a random route of variable length from the start node to either of the
    base stations. This is done by checking valid nodes (those with connectivity) while
    recursively traversing the such space upto to a point where either base stations is
    reached
    returns: the route with an end-to-end transmission rate > 0 and the transmission rate
    """
    # start creating path from origin
    route = [origin]
    current_node = route[-1]
    while current_node not in list(x_y_data.keys())[-2:]:  
        valid_indices = [loc + 1 for loc in range(len(bandwidth_data[current_node - 1])) if bandwidth_data[current_node - 1][loc] > 0 and (loc + 1) not in route]
        if len(valid_indices) == 0:
            # if number of valid neighbors (without repitition) is 0, try again
            route = [origin]
        else:
            # choose next index based on current nodes neighbors
            route.append(choice(np.array(valid_indices), 1, replace=False)[0])
        # set the last node as the current node
        current_node = route[-1]
    # calculate the route cost
    route_cost = bandwidth_latency_calculation(route)

    return route, route_cost

In [203]:
def bandwidth_latency_calculation(route):
    """
    bandwidth_latency_calc(routes)
    calculates the end-to-end transmission rate for the route provided
    the function also stores
    route: the path whose end-to-end transmission is being calculated
    returns: the absolute path cost of the route
    """
    global sim_anneal_data, solutions
    route_costs = []

    # var for storing all path bandwidths
    path_bandwidths = []
    path_latency = 0
    # we do -1 because we need to get to second last node where we will check cost between it and last node
    for i in range(len(route) - 1):
        # get path distance
        path_distance = calculate_distance(x_y_data[route[i]], x_y_data[route[i + 1]])
        # get bandwidth between node and its next node and append it to path bandwidths var
        path_bandwidth = convert_distance_bandwidth(path_distance)
        path_latency += node_to_node_latency
        path_bandwidths.append(path_bandwidth)
    # compute end-to-end bandwidth rate
    end_to_end_bandwidth = min(path_bandwidths)
    # step latency down to 0 if bandwidth is 0
    if end_to_end_bandwidth == 0:
        absolute_path_cost = 0
    else:
        # update the map visualization data and best solution data for recording in the solutions file
        absolute_path_cost = round(absolute_cost_function(end_to_end_bandwidth, path_latency), 3)
        if len(sim_anneal_data['best_path']) == 0:
            # if there is no best solution yet, add this as one
            solutions[0] = {"source node": f"Node-{route[0]}"}
            routing_path = ''
            for idx, sensor in enumerate(route[1:]):
                routing_path += f"(Node-{sensor}, {path_bandwidths[idx]} Mbps),"
            solutions[0]["routing path"] = routing_path
            solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
            solutions[0]["end-to-end latency"] = f"{path_latency} ms"
            solutions[0]["absolute path cost"] = f"{absolute_path_cost}"
            sim_anneal_data['best_path'][str(list(route))] = {'end_to_end_bandwidth': end_to_end_bandwidth,
                                                 'path_latency': path_latency,
                                                 'absolute_cost': absolute_path_cost
                                                }
        else:
            best_path = sim_anneal_data['best_path'][list(sim_anneal_data['best_path'].keys())[0]]
            if absolute_path_cost > best_path['absolute_cost']:
                '''
                if current solution is better than best solution, add best solution as 
                possible solution and set the current solution as the best solution
                '''
                solutions[0] = {"source node": f"Node-{route[0]}"}
                routing_path = ''
                for idx, sensor in enumerate(route[1:]):
                    routing_path += f"(Node-{sensor}, {path_bandwidths[idx]} Mbps),"
                solutions[0]["routing path"] = routing_path
                solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
                solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                solutions[0]["absolute path cost"] = f"{absolute_path_cost}"
                sim_anneal_data['possible_paths'][list(sim_anneal_data['best_path'].keys())[0]] = {
                                                 'end_to_end_bandwidth': best_path['end_to_end_bandwidth'],
                                                 'path_latency': best_path['path_latency'],
                                                 'absolute_cost': best_path['absolute_cost']
                                                }
                sim_anneal_data['best_path'].pop(list(sim_anneal_data['best_path'].keys())[0])
                sim_anneal_data['best_path'][str(list(route))] = {
                                                 'end_to_end_bandwidth': end_to_end_bandwidth,
                                                 'path_latency': path_latency,
                                                 'absolute_cost': absolute_path_cost
                                                }
            else:
                # if current solution is not better than best solution add it as possible solution
                sim_anneal_data['possible_paths'][str(list(route))] = {
                                                 'end_to_end_bandwidth': end_to_end_bandwidth,
                                                 'path_latency': path_latency,
                                                 'absolute_cost': absolute_path_cost
                                                }

    return absolute_path_cost

In [204]:
def simulated_annealing(origin, init_temp=100, final_temp=0.1, cooling_rate=0.95, improvement_checker_count=50):
    """
    simulated_annealing(origin, init_temp=100, final_temp=0.1, cooling_rate=0.95)
    the simulated annealing algorithm main method that gets the shortest path to 
    either of the base stations.
    origin: the start node (sensor) from which the search is being conducted
    init_temp: the start temperature defined for the simulated annealing algorithm (default: 100)
    final_temp: the final temperature defined for the simulated annealing algorithm (default: 0.1)
    cooling_rate: the temperature cooling rate for the algorithm for the algorithm (default: 0.95)
    improvement_checker_count: the number of iterations to go through before checking if there is
    any sensible improvement in the path quality (defult: 50)
    returns: sim_anneal_data and solutions variables with updates values
    """
    # re-initialize the 'sim_anneal_data' and 'solutions' global variables every time the function is called
    global sim_anneal_data, solutions
    sim_anneal_data = {'best_path': {},
                       'possible_paths': {}}
    solutions = [{}]
    bandwidth_data = calculate_route_costs(x_y_data)
    current_path, current_path_cost = generate_route(origin, bandwidth_data)     
    current_temp = init_temp
    i = 1
    # create variable that stores path costs that will keep track of significant improvements
    path_costs = [current_path_cost]
    # Iterate until the temperature is below the final temperature
    while current_temp > final_temp:
        path, path_cost = generate_route(origin, bandwidth_data)
        path_costs.append(path_cost)
        # Calculate the change in solution quality
        delta = path_cost - current_path_cost  
        if delta >= 0:
            # If the new path is better, accept it as the current path
            current_path = path
            current_path_cost = path_cost
            if i % improvement_checker_count == 0:
                # get different between the current performance and the -nth performance
                improvement = path_costs[i - improvement_checker_count]
                # if the change is less than one, break the loop
                if improvement < 1:
                    break
        else:
            # If the new path is worse, accept it with a certain probability
            probability = math.exp(-delta / current_temp)
            if random.random() < probability:
                current_path = path
                current_path_cost = path_cost
        # Decrease the temperature according to the cooling rate
        current_temp *= cooling_rate
        i += 1

    # Return the final solution and its quality, and the list of temperatures
    return sim_anneal_data, solutions

In [205]:
#simulated_annealing(101)

({'best_path': {'[101, 9, 70, 152]': {'end_to_end_bandwidth': 1,
    'path_latency': 90,
    'absolute_cost': 2.667}},
  'possible_paths': {'[101, 148, 122, 91, 143, 129, 105, 77, 72, 62, 90, 14, 27, 66, 137, 88, 75, 31, 149, 117, 65, 130, 80, 51, 144, 34, 107, 64, 7, 50, 63, 60, 39, 32, 121, 109, 71, 11, 147, 48, 13, 67, 140, 78, 84, 118, 92, 131, 38, 128, 4, 2, 83, 18, 19, 85, 116, 139, 81, 115, 28, 43, 37, 36, 120, 45, 138, 10, 136, 74, 103, 61, 73, 112, 135, 41, 35, 79, 113, 46, 95, 58, 89, 25, 47, 82, 12, 98, 102, 142, 119, 29, 30, 24, 23, 54, 59, 127, 100, 17, 6, 87, 134, 52, 42, 8, 86, 106, 68, 69, 16, 5, 99, 152]': {'end_to_end_bandwidth': 1,
    'path_latency': 3390,
    'absolute_cost': 1.044},
   '[101, 150, 127, 20, 114, 96, 24, 102, 110, 79, 143, 47, 48, 128, 13, 4, 105, 138, 139, 89, 19, 16, 7, 119, 113, 130, 30, 25, 34, 68, 29, 122, 148, 124, 92, 28, 72, 15, 62, 115, 41, 97, 81, 120, 61, 49, 84, 112, 90, 57, 53, 77, 131, 35, 98, 66, 145, 118, 74, 37, 36, 133, 52, 59, 134